In [1]:
from pylab import *
from scipy import *
from matplotlib.colors import LogNorm
from matplotlib import colors
from IPython.display import Image
from scipy.integrate import quad
from scipy import interpolate
from scipy.special import jn, gamma
from scipy.misc import derivative
from scipy.integrate import nquad
import camb
from camb import model, initialpower

In [9]:
### constants
c = 299792.458#km/s
Gnewton = 6.674e-8#cgs cm^3/g/s^2

## cosmology WMAP9
h = 0.7
H0 = h*100
oc = 0.236
ob = 0.046
om = ob+oc
ol = 1-om#0.718
ns = 0.9646
s8 = 0.817
#betaD = finterp(0.0)/bD
## test array
PI_arr=logspace(-0.5, 1, 50)
rp_arr = [1.0, 2.0, 5.0, 10.0, 20.0, 40.0]

## small functions
H = lambda z: H0*sqrt(om*(1+z)**3+ol)
H_inv = lambda z:  1/(H0*sqrt(om*(1+z)**3+ol))
Hcgs = lambda z: H(z)*3.24e-20
DC = lambda z: c*quad(H_inv, 0, z)[0]
W_fcn = lambda z: (pz(z) / DC(z))**2 / c * H(z) # dchi/dz = c/H
rho_cz = lambda z: 0.375*Hcgs(z)**2/pi/Gnewton

Wnorm = quad(W_fcn, zmin, zmax) [0]
W_arr = array([W_fcn(iz)/Wnorm for iz in zarr])
W = interpolate.interp1d(zarr,W_arr,bounds_error=0,fill_value=0.)

In [3]:
data_mean = loadtxt('/Users/jia/weaklensing/gplus/fulle_bins2D_cross_jk_final.dat')[:,5].reshape(25,-1).T
rp_bins = loadtxt('/Users/jia/weaklensing/gplus/fulle_bins2D_cross_jk_final.dat')[:,0].reshape(25,-1).T
pi_bins = loadtxt('/Users/jia/weaklensing/gplus/fulle_bins2D_cross_jk_final.dat')[:,1].reshape(25,-1).T

## make a reflection for negative r 
data_mean2=concatenate([data_mean,data_mean])
rp_bins2=concatenate([rp_bins,-rp_bins])
pi_bins2=concatenate([pi_bins,pi_bins])

icolors=['yellow','cyan','red','blue','mediumpurple']
icolors_deep=['darkorange','deepskyblue','maroon','midnightblue','purple']
bounds=logspace(-3,0,6)

In [6]:
zmin,zmax = 0.16, 0.36
zcenter,dndz = load('dndz_lowz.npy')
pz = interpolate.interp1d(zcenter,dndz,bounds_error=0,fill_value=0.)

In [7]:
### power spectrum from camb
zarr = linspace(zmin, zmax, 21)
pars = camb.CAMBparams()
pars.set_cosmology(H0=70, ombh2=ob*h**2, omch2=om*h**2)#omch2=oc*h**2)
pars.InitPower.set_params(ns=0.965)
pars.set_matter_power(redshifts=zarr, kmax=100.0)
results = camb.get_results(pars)
########### NL power spectrum ########
pars.NonLinear = model.NonLinear_both
#pars.NonLinear = model.NonLinear_none  #linear
results.calc_power_spectra(pars)
kh_nonlin, z_nonlin, pk_nonlin = results.get_matter_power_spectrum(minkh=1e-4, maxkh=100.0, npoints = 150)

Note: redshifts have been re-sorted (earliest first)


In [11]:
########## marko functions

kPk = pk_nonlin[0]*kh_nonlin
Nk = len(kPk)
kPk_fft = fft(kPk) ## FFT
klogstep = log(kh_nonlin[1]/kh_nonlin[0])

########## now compare how accurate marko's integration is
kmin,kmax=kh_nonlin[[0,-1]]
ifreq = fftfreq(Nk)##
nu_arr = 1j*2*pi*fftfreq(Nk, d=klogstep) ## the argument goes into k^nu_n, now nu_n is nu_arr
cn = kPk_fft*kmin**(-nu_arr) 

#fun_marko = lambda rp, PI, nu: rp**2/(rp**2+PI**2)**(3.0+nu/2.0)*cos(pi*nu/2.0)*gamma(3.0+nu)*( (nu-3.0)*(rp**2+PI**2)+ (rp**2-(3+nu)*PI**2*betaD))/(nu-3.0)/(nu**2-1)
fun_marko = lambda rp, PI, nu: rp**2/(rp**2+PI**2)**(2.0+nu/2.0)*cos(pi*nu/2.0)*gamma(3.0+nu)/(1-nu**2)
xi_int_marko = lambda rp, PI: sum(cn*fun_marko(rp, PI, nu_arr))/len(nu_arr)
marko_arr = array([[xi_int_marko(irp, iPI) for iPI in PI_arr] for irp in rp_arr])